In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf=SparkConf().setAppName("BDProject").setMaster("local[4]")

In [4]:
sc=SparkContext(conf=conf)

In [74]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, StringType, DateType

In [6]:
spark = SparkSession.builder \
    .appName('ss') \
    .master('local[4]') \
    .getOrCreate()

Importujemy rozpakowane dane tekstowe

In [7]:
pathUsers="travel.stackexchange.com\\Users.xml"

In [8]:
rawUsers=sc.textFile(pathUsers)

In [9]:
rawUsers.filter(lambda x: x.startswith('  <row Id=')).count()

5063

In [10]:
rawUsers.take(4)

['<?xml version="1.0" encoding="utf-8"?>',
 '<users>',
 '  <row Id="-1" Reputation="1" CreationDate="2011-06-21T15:16:44.000" DisplayName="Community" LastAccessDate="2011-06-21T15:16:52.063" Location="on the server farm" AboutMe="&lt;p&gt;Hi, I\'m not really a person.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I\'m a background process that helps keep this site clean!&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I do things like&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;&lt;li&gt;Randomly poke old unanswered questions every hour so they get some attention&lt;/li&gt;&#xA;&lt;li&gt;Own community questions and answers so nobody gets unnecessary reputation from them&lt;/li&gt;&#xA;&lt;li&gt;Own downvotes on spam/evil posts that get permanently deleted&lt;/li&gt;&#xA;&lt;li&gt;Own suggested edits from anonymous users&lt;/li&gt;&#xA;&lt;li&gt;&lt;a href=&quot;http://meta.stackexchange.com/a/92006&quot;&gt;Remove abandoned questions&lt;/a&gt;&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;" Views="30" UpVotes="21" DownVotes="17" Age="7" AccountId=

Tworzymy funkcję czyszczącą RDDs do listy Row'ów

In [63]:
def rddToDf(RDDIn):
    def preCleanUp(RDDIn):
        def cleaning(listIn):
            listOut=[]
            for x in listIn:
                if listIn.index(x)%2==0:
                    listOut.append(x.replace(' ','').replace('<','').replace('=',''))
                else:
                    listOut.append(x)
            return {listOut[i]:listOut[i+1] for i in range(0,len(listOut)-1,2)}
        def casting(dictIn):
            '''formatujemy datę i tekst'''
            def textMining(string):
                import re
                listXmlChar={'&lt;':'', '&amp':'&','&gt;':'','&quot;':'"','&apos;':'\'','&#xA;':''}
                y=re.sub('&lt;.*?&gt;','',string)
                for x in listXmlChar.keys():
                    z=y.replace(x,listXmlChar[x])
                    y=z
                return z
            from datetime import datetime
            for x in dictIn.keys():
                if x.find('Date')>0:
                    dictIn[x]=datetime.strptime(dictIn[x][0:-4],'%Y-%m-%dT%H:%M:%S')
                else:
                    dictIn[x]=textMining(dictIn[x])
            return dictIn
        return RDDIn.filter(lambda x: x.startswith('  <row Id=')).map(lambda x: x.split('"')).map(cleaning).map(casting)
    dictRDD=preCleanUp(RDDIn)
    listOfColums=dictRDD.flatMap(lambda x: list(x.keys())).distinct().collect()
    def newDict(dictIn):
        for col in listOfColums:
            if col not in dictIn.keys():
                dictIn[col]=None
        return dictIn
    return dictRDD.map(newDict).map(lambda x: Row(**x)).toDF(sampleRatio=100)

In [64]:
usersDF=rddToDf(rawUsers)

In [72]:
usersDF.select('AboutMe', 'DisplayName').where(usersDF.AboutMe!='').show()

+--------------------+----------------+
|             AboutMe|     DisplayName|
+--------------------+----------------+
|Hi, I'm not reall...|       Community|
|Developer on the ...|    Geoff Dalgas|
|I am the Architec...|     Nick Craver|
|co-founder of Air...|          Emmett|
|Stack Overflow Va...|  Kevin Montrose|
|I am a 2009 gradu...|   Jimmy Sawczuk|
|Developer by Trad...|        Seanland|
|I'm a django deve...|    silent1mezzo|
|I'm a software de...|     Darren Kopp|
|I'm a Web Develop...|      daybreaker|
|I'm a software de...|    Matthew Read|
|Java Slave.. hem ...|         cerealk|
|.Net Developer si...|           VMAtm|
|Former Stack Exch...|Rebecca Chernoff|
|A Dutch guy who l...|          jeroen|
|President Fog Cre...|   Michael Pryor|
|Web/server/app so...|          Nicole|
|          wondering.|       victoriah|
|Game, web, and pr...|         justinl|
|I love to travel,...|             Jan|
+--------------------+----------------+
only showing top 20 rows



In [78]:
usersDF.withColumn('AgeInt',usersDF.Age.cast("integer")).select("AgeInt").show(4)

+------+
|AgeInt|
+------+
|     7|
|    41|
|    33|
|    32|
+------+
only showing top 4 rows



In [16]:
listXmlChar={'&lt;':'<', '&amp':'&','&gt;':'>','&quot;':'"','&apos;':'\''}

In [80]:
usersDF.select('Age').describe().show()

+-------+------------------+
|summary|               Age|
+-------+------------------+
|  count|              1978|
|   mean|34.353387259858444|
| stddev|11.293929737483065|
|    min|               103|
|    max|                98|
+-------+------------------+



In [82]:
usersDF.select('Age').where(usersDF.Age.isNull()).count()

3085

In [98]:
usersDF.groupBy('Age').count().orderBy('count',ascending=False).show(5)

+----+-----+
| Age|count|
+----+-----+
|null| 3085|
|  29|  134|
|  30|  133|
|  31|  105|
|  32|  105|
+----+-----+
only showing top 5 rows

